In [9]:
import json
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn import linear_model
import sklearn
import numpy
import random
import gzip
import math
import warnings
warnings.filterwarnings('ignore')
import numpy          # linear algebra
import urllib         # load data from the web
import scipy.optimize # optimization routines
import random         # random number generation
import ast
import string
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
def readGz(path):
    i = 0
    for l in gzip.open(path, 'rt'):
        if i < 500000:
            yield eval(l)
            i += 1

In [11]:
data = []

for d in readGz("ratebeer.json.gz"):
    data.append(d)

In [12]:
data[1]['beer/style']

'India Pale Ale &#40;IPA&#41;'

In [13]:

data50000 = []
top5 = ['India Pale Ale &#40;IPA&#41','Pale Lager','Belgian Strong Ale','Imperial Stout','Imperial/Double IPA']
# organize by top 5, beer/styles, we only want 50k datapoints within this criteria
for i in range(0,50000):
    if (data[i]['beer/style'] in top5):
        data50000.append(data[i]) # add that data point (review)

In [14]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data50000:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akhil3rain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
stopWords = stopwords.words('english')

tempList = []
for freq,wrd in counts:
    #print(freq,wrd)
    if (wrd not in stopWords):
        tempList.append((freq,wrd)) # using .remove causes issues
    
#print(tempList)
counts = tempList
#print(counts)


In [18]:
NW = 1000 # dictionary size
words = [x[1] for x in counts[:NW]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if w in wordSet:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    # feat[0]
    return feat

In [19]:
X = [feature(d) for d in data50000]
y = [d['beer/style'] for d in data50000] # beer/beerId
# the key to model.fit running faster is changing to bag of words y-reference (beer/style etc...)

# at c=10
# the lower size the training data parameters are, the lower the accuracy
a = 9     # 5,10 gives acc = 0.0? # 1,5 gives 0.0159   # 1,10 gives 0.032711111111111114
b = 10    # 1,100 0.0008484848484848485 # 8,10 gives 0.0027 # 9,10 gives 0.0

Xtrain = X[:a*len(X)//b] # larger sets just don't run fast at all (maybe because i have shit pc) #5k is fast asf
ytrain = y[:a*len(y)//b]
Xvalid = X[a*len(X)//b:]
yvalid = y[a*len(y)//b:]

In [20]:
len(Xtrain)

7101

In [21]:
len(Xtrain)

7101

In [28]:
from sklearn.naive_bayes import GaussianNB
import math
#Create a Gaussian Classifier
model = GaussianNB(var_smoothing = math.log(2))

# Train the model using the training sets
model.fit(Xtrain,ytrain)

#Predict Output
predicted= model.predict(Xvalid) 

In [30]:
correct = predicted == yvalid
sum(correct) / len(correct)

0.8694550063371356